In [ ]:
import pandas as pd
import json
import warnings
warnings.simplefilter('ignore')

In [ ]:
# read the Excel file into a pandas dataframe
df = pd.read_excel('Hotels.xlsx')

## Scrape the Country name from booking.com

In [ ]:
#Country Scraping
from bs4 import BeautifulSoup
import requests
import urllib.parse
import time
    # set the search query
def extract_country(query):
    try:
        # encode the query for use in the Google search URL
        query_url_encoded = urllib.parse.quote_plus(query)

        # make a request to the Google search results page
        url = f"https://www.google.com/search?q={query_url_encoded}"
        response = requests.get(url)
        # parse the HTML content of the search results page using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        key_word='booking'
        # find the first search result link
        search_results = soup.find_all('a')
        for link in search_results:
            href = link.get('href')
            if href and 'booking' in href:
                # extract the URL from the href attribute
                url = href.split('=')[1]
                url = urllib.parse.unquote_plus(url.split('&')[0])
                break
        response = requests.get(url)
        # parse the HTML content of the search results page using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        address = soup.find('script', {'type': 'application/ld+json'})
        if (address==None):
            country='Not Found'
            return country
        address = address.text
        address_dict = json.loads(address)['address']
        # extract the country from the address object
        country = address_dict['addressCountry']
        return country
    except Exception:
        country='Not Found'
        return country


In [ ]:
#extract the hotel names only into new df
df2=df['Hotel Name'].to_frame()

In [ ]:
# define the range of numbers
start = 0
stop = 2500

# define the batch size
batch_size = 16

# define an empty list to store the batches
batches = []

# iterate over the batches
for i in range(start, stop, batch_size):
    # get the batch of numbers
    batch = range(i, min(i + batch_size, stop))

    # append the batch to the list
    batches.append(list(batch))

In [ ]:

# split the DataFrame into batches of size 32
batches = np.array_split(df2, len(df2) // 32 + 1)

# create a list of DataFrames
df_list = [batch.reset_index(drop=True) for batch in batches]

In [ ]:
#split the df into 79 batches each one has 32
len(df_list)

79

In [ ]:
#try the function
extract_country(df['Hotel Name'][8])

'USA'

In [ ]:
#apply threading and make delay to avoid been blocked
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=8)
import time
# apply the extract_country() function to the 'Hotel Name' column using multithreading and store the result in a new column 'Country'
for i in range(len(df_list)):
    df_list[i]['Country'] = list(executor.map(extract_country, df_list[i]['Hotel Name']))
    time.sleep(10)


IndexError: list index out of range

In [ ]:
#concatenate our batches into one df again
c_df = pd.concat(df_list[:], ignore_index=True)


In [ ]:
#merge the country to our original data df
final_df = pd.merge(df,c_df , on='Hotel Name')


In [ ]:
final_df.head()

,Code,Hotel Name,Roomnights,Sale,Cost,Date,Country
0,CODE|87851,Best Western Premier Al Ahsa Grand Hotel & Suites,57,1140,1140,"01, 2019",Saudi Arabia
1,CODE|87816,Best Western Premier Airporthotel Fontane Berlin,1935,50310,38700,"08, 2019",Germany
2,CODE|87781,Best Western Premier Agung Resort Ubud,1897,37940,34146,"01, 2019",Indonesia
3,CODE|87746,Best Western Posada Freeman,1294,38820,25880,"06, 2019",Mexico
4,CODE|87711,Best Western Pony Soldier Inn & Suites,828,22356,13248,"12, 2019",USA


In [ ]:
#save the data after scraping into csv file
final_df.to_csv('final.csv', index=False)